In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset, Dataset
from transformers import ElectraTokenizer, ElectraForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import torch
import re

In [ ]:
df = pd.read_csv('labeled_data.csv')

In [ ]:
def preprocess_data(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    return text

In [ ]:
df['processed_tweet'] = df['tweet'].apply(preprocess_data)
label_map = {0: 0, 1: 1, 2: 2}
df['label'] = df['class'].map(label_map)

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(df['processed_tweet'], df['label'], test_size=0.2, random_state=42, stratify=df['label'])

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
train_texts_tfidf = tfidf.fit_transform(train_texts).toarray()

In [ ]:
smote = SMOTE(random_state=42)
train_texts_resampled, train_labels_resampled = smote.fit_resample(train_texts_tfidf, train_labels)

In [ ]:
train_texts_resampled = tfidf.inverse_transform(train_texts_resampled)
train_texts_resampled = [" ".join(row) for row in train_texts_resampled]

In [ ]:
train_data = Dataset.from_dict({'text': train_texts_resampled, 'label': train_labels_resampled})
test_data = Dataset.from_dict({'text': test_texts, 'label': test_labels})

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=128)

In [ ]:
train_data = train_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/46056 [00:00<?, ? examples/s]

Map:   0%|          | 0/4957 [00:00<?, ? examples/s]

In [ ]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
model = ElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator', num_labels=3)

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    roc_auc = roc_auc_score(np.eye(3)[labels], logits, multi_class='ovo', average='weighted')
    return {"accuracy": acc, "f1": f1, "roc_auc": roc_auc}

In [ ]:
training_args = TrainingArguments(
    output_dir="./electra_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-35-e7dec7d42aac>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()
results = trainer.evaluate()
print(results)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc
1,0.378900,0.601639,0.767198,0.817950,0.946494
2,0.282400,0.502245,0.818237,0.851447,0.943784


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc
1,0.378900,0.601639,0.767198,0.817950,0.946494
2,0.282400,0.502245,0.818237,0.851447,0.943784
3,0.251800,0.539197,0.805931,0.842801,0.940554


{'eval_loss': 0.5022446513175964, 'eval_accuracy': 0.8182368367964494, 'eval_f1': 0.8514466181654626, 'eval_roc_auc': 0.9437843432868422, 'eval_runtime': 348.1169, 'eval_samples_per_second': 14.239, 'eval_steps_per_second': 0.445, 'epoch': 3.0}


In [ ]:
def test_input(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
    outputs = model(**inputs)
    probabilities = torch.softmax(outputs.logits, dim=1).detach().cpu().numpy()[0]
    predicted_class = np.argmax(probabilities)
    class_labels = {0: 'Hate Speech', 1: 'Offensive Language', 2: 'Neither'}
    print(f"Input: {text}")
    print(f"Predicted Class: {class_labels[predicted_class]}")
    print("Class Probabilities:")
    for cls, prob in zip(class_labels.values(), probabilities):
        print(f"  {cls}: {prob:.4f}")

In [ ]:
example_text = "You're worthless"
test_input(example_text)

Input: You're worthless
Predicted Class: Hate Speech
Class Probabilities:
  Hate Speech: 0.5651
  Offensive Language: 0.2269
  Neither: 0.2080
